In [1]:
import pandas as pd
import ast

c:\Users\n9810200\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\n9810200\AppData\Local\Temp/ipykernel_16220/4135976894.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
denominator = 1600 #same as the number of unique o2o relationships

##### Dirigo

In [3]:
dirigo_e2o = pd.read_csv('../Dirigo/OCEL_E2O.csv')
dirigo_e2o

,Event_id,Object_id,E2O_qualifier
0,Lodge_Pcp44,Pcp44,Pickup plan being lodged
1,Lodge_Pcp44,Cr2,Cargo being scheduled for pickup
2,Lodge_Pcp87,Pcp87,Pickup plan being lodged
3,Lodge_Pcp87,Cr9,Cargo being scheduled for pickup
4,Lodge_Pcp48,Pcp48,Pickup plan being lodged
...,...,...,...
1595,eval_trs_exit_tr49_Pcp70,tr49,Truck being evaluated for exit
1596,weigh_loaded_truck_Pcp70_tr29,tr29,Loaded truck getting weighed
1597,print_w_ticket_Pcp70_tr29,tr29,Truck receiving weighing ticket
1598,input_tally_sheet_Pcp70_tr29,tr29,Truck being recorded on tally sheet


In [4]:
qualified_e2o_dirigo = dirigo_e2o['E2O_qualifier'].notna() & (dirigo_e2o['E2O_qualifier'].str.strip() != '')
num_qualified_dirigo = qualified_e2o_dirigo.sum()
num_qualified_dirigo

1600

In [5]:
e2oscr_dirigo = num_qualified_dirigo / denominator
print(f"E2O Qualifier Completeness Ratio (Dirigo): {e2oscr_dirigo:.2%}")

E2O Qualifier Completeness Ratio (Dirigo): 100.00%


##### ACEL

In [6]:
acel_events = pd.read_csv('../ACEL/ACEL_Events.csv')


In [7]:
e2o_qualifier_tuples = []

for _, row in acel_events.iterrows():
    event_id = row['Event_id']
    object_changes_raw = row.get('ObjectChanges')

    if pd.notna(object_changes_raw):
        try:
            object_changes = ast.literal_eval(object_changes_raw)

            for change in object_changes:
                if change.get('Attribute') == 'lifecycle':
                    object_id = change.get('ObjectId')
                    lifecycle_value = change.get('NewValue')

                    if object_id and lifecycle_value:
                        e2o_qualifier_tuples.append((event_id, object_id, lifecycle_value))
        except Exception:
            continue  

In [8]:
df_e2o_qualifier = pd.DataFrame(
    e2o_qualifier_tuples,
    columns=['Event_id', 'Object_id', 'E2O_qualifier']
).drop_duplicates()
df_e2o_qualifier

,Event_id,Object_id,E2O_qualifier
0,Lodge_Pcp44,Pcp44,lodged
1,Lodge_Pcp44,Cr2,scheduled
2,Lodge_Pcp87,Pcp87,lodged
3,Lodge_Pcp87,Cr9,scheduled
4,Lodge_Pcp48,Pcp48,lodged
...,...,...,...
1595,eval_trs_exit_tr49_Pcp70,tr49,released
1596,weigh_loaded_truck_Pcp70_tr29,tr29,weigh_loaded
1597,print_w_ticket_Pcp70_tr29,tr29,ticket_issued
1598,input_tally_sheet_Pcp70_tr29,tr29,tally_recorded


In [9]:
qualified_e2o_acel = df_e2o_qualifier ['E2O_qualifier'].notna() & (df_e2o_qualifier ['E2O_qualifier'].str.strip() != '')
num_qualified_acel = qualified_e2o_acel.sum()
num_qualified_acel

1600

In [10]:
e2oscr_acel = num_qualified_acel / denominator
print(f"E2O Qualifier Completeness Ratio (ACEL): {e2oscr_acel:.2%}")

E2O Qualifier Completeness Ratio (ACEL): 100.00%
